In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'

# data info
df_info = pd.read_csv('data_subc_sig_v1.csv')
df_info = df_info[df_info.id_person < 30]

person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))

# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_time = int(np.max(df_info['len']))
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# make data generator
def gen_csi(df_info,id_num,len_num):
    for file in set(df_info.id.values):
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
        abs_sub = np.mean(np.abs(data1),axis=(0,2,3))
        data1_norm = data1/abs_sub[np.newaxis,:,np.newaxis,np.newaxis]

        data1_abs = np.abs(data1_norm)
        data1_ph = np.angle(data1_norm)
        
        data1_diff = np.abs(data1)[:500]
        
        # differentiation
        #data1_diff = np.diff(data1_abs,axis=0)
        
        # zero pad
        pad_len = len_num - data1_diff.shape[0]
        data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # Label
        id_key = df_info[df_info.id==file]['id_person'].values[0].astype('int')
        data1_y = dict_id[id_key]

        x_input = data1_pad.reshape([-1,len_num,30,6]).astype('float32')
        y_input = tf.keras.utils.to_categorical(data1_y, no_classes).reshape([1,-1])
        
        yield(x_input / max_value,y_input)

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
generator = gen_csi(df_info,no_classes,csi_time)
list_data = []
list_label = []
for i in range(len(df_info)):
    data,label = next(generator)
    list_data.append(data)
    list_label.append(label)

StopIteration: 

In [5]:
list_data1 = [row[-1,:500,:,:] for row in list_data]
list_data1[0].shape

(500, 30, 6)

In [6]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'

# data info
df_info = pd.read_csv('data_subc_sig.csv')
# parameters
no_classes = np.max(df_info['id_person'])
epochs = 10
batch_size = 10
#image_height = int(np.max(df_info['len']))
image_height,image_width = 500,30
input_shape = (image_height, image_width, 6)

In [8]:
data_read = np.array(list_data1)
data_read.shape

(836, 500, 30, 6)

In [9]:
#filter outliers
xs_med = np.median(np.abs(data_read))
xs_std = np.std(np.abs(data_read))
xs_th = xs_med+xs_std

data_read_f =  data_read

In [10]:
data_y = np.array(list_label)

In [12]:
data_y1 = np.array([np.argmax(row,1) for row in list_label]).squeeze()
data_y1.shape
y_train =  tf.keras.utils.to_categorical(data_y1, no_classes)
y_train.shape

(836, 107)

In [13]:
data_read.shape
xs_th

0.3980647

In [ ]:
#models
def simple_cnn(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [ ]:
x_train = data_read_f
# training
simple_cnn_model = simple_cnn(input_shape)
simple_cnn_model.fit(data_read, y_train, batch_size, epochs)

In [ ]:
y_train[0].shape